<a href="https://colab.research.google.com/github/ReemAlsharabi/CryptoRobustTraining/blob/main/crypten_mnist_fgsm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### deps

Don't pip install crypten directly because you'll get [this error](https://github.com/facebookresearch/CrypTen/issues/466)

In [ ]:
!pip install torch>=1.7.0 torchvision>=0.9.1 omegaconf>=2.0.6 onnx>=1.7.0 pandas>=1.2.2 pyyaml>=5.3.1 tensorboard future scipy>=1.6.0 wget

In [ ]:
!pip install --no-deps crypten

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.9/259.9 kB 1.6 MB/s eta 0:00:00


In [ ]:
import crypten
import torch
import torchvision
import matplotlib.pyplot as plt
import wget

%matplotlib inline

crypten.init()
torch.set_num_threads(1)

In [ ]:
def set_seed(seed):
    # random.seed(seed)
    # np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
set_seed(42)

In [ ]:
# device = 'cuda' if torch.cuda.is_available() else 'cpu'

### data

In [ ]:
data_train = torchvision.datasets.MNIST(root='/tmp/data',
                                           train=True,
                                           transform=torchvision.transforms.ToTensor(),
                                           download=True)

data_test = torchvision.datasets.MNIST(root='/tmp/data',
                                           train=False,
                                           transform=torchvision.transforms.ToTensor(),
                                           download=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 127947705.43it/s]

Extracting /tmp/data/MNIST/raw/train-images-idx3-ubyte.gz to /tmp/data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 132099993.26it/s]


Extracting /tmp/data/MNIST/raw/train-labels-idx1-ubyte.gz to /tmp/data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 36942680.24it/s]

Extracting /tmp/data/MNIST/raw/t10k-images-idx3-ubyte.gz to /tmp/data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 5681636.97it/s]


Extracting /tmp/data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /tmp/data/MNIST/raw



In [ ]:
def take_samples(data, n_samples=1000):
    """Returns images and labels based on sample size"""
    images, labels = [], []

    for i, d in enumerate(data):
        if i == n_samples:
            break
        image, label = d
        images.append(image)
        label_one_hot = torch.nn.functional.one_hot(torch.tensor(label), 10)
        labels.append(label_one_hot)

    images = torch.cat(images)
    labels = torch.stack(labels)
    return images, labels

In [ ]:
images_train, labels_train = take_samples(data_train, n_samples=100)
print(images_train.shape)
print(labels_train.shape)

torch.Size([100, 28, 28])
torch.Size([100, 10])


In [ ]:
images_train_enc = crypten.cryptensor(images_train, requires_grad=True)
labels_train_enc = crypten.cryptensor(labels_train, requires_grad=True)

# test set
images_test, labels_test = take_samples(data_test, n_samples=20)
images_test_enc = crypten.cryptensor(images_test, requires_grad=True)
labels_test_enc = crypten.cryptensor(labels_test, requires_grad=True)

In [ ]:
images_train_enc[0]

MPCTensor(
	_tensor=tensor([[    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
     

# Encrypted model

In [ ]:
class CNN_Enc(crypten.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = crypten.nn.Conv2d(1, 32, 3, 1)
        self.conv2 = crypten.nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = crypten.nn.Dropout2d(0.25)
        self.dropout2 = crypten.nn.Dropout2d(0.5)
        self.fc1 = crypten.nn.Linear(64*12*12, 128)
        self.fc2 = crypten.nn.Linear(128, 10)

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.conv1(x)
        x = x.relu()
        x = self.conv2(x)
        x = x.relu()
        x = x.max_pool2d(2)
        x = self.dropout1(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = x.relu()
        x = self.dropout2(x)
        x = self.fc2(x)
        return x

The model has to be encrypted so we train it on CrypTensors. [See why](https://github.com/facebookresearch/CrypTen/issues/279)

In [ ]:
model = CNN_Enc().encrypt()

In [ ]:
x = images_train_enc[0].unsqueeze(0)
print(x.shape)
model(x)

torch.Size([1, 28, 28])


MPCTensor(
	_tensor=tensor([[ 9263, -5638, -3432,  2022,  -209,  5508,  3322, -4478, -6147,  2496]])
	plain_text=HIDDEN
	ptype=ptype.arithmetic
)

### encrypted training

**How does CrypTen training differ from PyTorch training?**

There are two main ways implementing a CrypTen training loop differs from a PyTorch training loop. We'll describe these items first, and then illustrate them with small examples below.

(1) Use one-hot encoding: CrypTen training requires all labels to use one-hot encoding. This means that when using standard datasets such as MNIST, we need to modify the labels to use one-hot encoding.

(2) Directly update parameters: CrypTen does not use the PyTorch optimizers. Instead, CrypTen implements encrypted SGD by implementing its own backward function, followed by directly updating the parameters, using SGD in CrypTen is very similar to using the PyTorch optimizers. [Implementing other optmizers](https://github.com/facebookresearch/CrypTen/issues/405) is also possible.

[source](https://github.com/facebookresearch/CrypTen/blob/f4cbdfc685d9064f45a5654dee9f3809f6d93e7f/tutorials/Tutorial_7_Training_an_Encrypted_Neural_Network.ipynb)

In [ ]:
def train_model(model, X, y, epochs=10, learning_rate=0.05):
    criterion = crypten.nn.CrossEntropyLoss()
    model.train()
    for epoch in range(epochs):
        model.zero_grad()
        output = model(X)
        loss = criterion(output, y)
        print(f"epoch {epoch} loss: {loss.get_plain_text()}")
        loss.backward()
        grads = X.grad
        # print(grads)
        model.update_parameters(learning_rate)
    return model, grads

# for inference use with crypten.no_grad():

In [ ]:
model, grads = train_model(model, images_train_enc, labels_train_enc, epochs=5, learning_rate=0.1)

epoch 0 loss: 2.3063507080078125
epoch 1 loss: 2.3021240234375
epoch 2 loss: 2.3012542724609375
epoch 3 loss: 2.2913665771484375
epoch 4 loss: 2.28167724609375


In [ ]:
prediction = model(images_test_enc[3].unsqueeze(0)).argmax()
prediction.get_plain_text()


tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [ ]:
labels_test[3] # ground truth

tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

### fgsm

In [ ]:
def fgsm_attack(input, epsilon, data_grad):
    sign_data_grad = data_grad.sign()
    perturbed_input = input + epsilon * sign_data_grad
    return perturbed_input

In [ ]:
perturbed_images = []
for i in range(len(images_test_enc)):
    input = images_test_enc[i]
    grad = grads[i]
    perturbed_input = fgsm_attack(input, 0.05, grad)
    perturbed_images.append(perturbed_input)

In [ ]:
adv_prediction = model(perturbed_images[3].unsqueeze(0)).argmax()
adv_prediction.get_plain_text()

tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

### auto-attack



---


not compatible with crypten tensors. Override AutoAttack class or re-implement the attacks?


---



In [ ]:
pip install git+https://github.com/fra31/auto-attack

In [ ]:
from autoattack import AutoAttack

In [ ]:
def forward_pass(input):
    logits = model(input)
    return logits

In [ ]:
epsilon = 0.01
adversary = AutoAttack(forward_pass(images_test_enc), norm='Linf', eps=epsilon, version='standard')

In [ ]:
x_adv = adversary.run_standard_evaluation(images_test_enc, labels_test_enc, bs=32)

In [ ]:
dict_adv = adversary.run_standard_evaluation_individual(images_test_enc, labels_test_enc, bs=32)

# Encrypting a Pre-trained Model
model pre-trained on plain data (encryption at test-time)

Even if we have a pre-trained model, CrypTen will require this structure as input.

### model

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class AliceNet(nn.Module):
    def __init__(self):
        super(AliceNet, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        out = self.fc1(x)
        out = F.relu(out)
        out = self.fc2(out)
        out = F.relu(out)
        out = self.fc3(out)
        return out
crypten.common.serial.register_safe_class(AliceNet)

In [ ]:
dummy_model = AliceNet() # provide a dummy model to tell CrypTen the model's structure
plaintext_model = torch.load('AliceNet.pth')
print(plaintext_model)

AliceNet(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)


In [ ]:
dummy_input = torch.empty((1, 784))
private_model = crypten.nn.from_pytorch(plaintext_model, dummy_input) # set up a CrypTen network from the PyTorch network

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



/usr/local/lib/python3.10/dist-packages/crypten/nn/onnx_converter.py:176: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  param = torch.from_numpy(numpy_helper.to_array(node))


In [ ]:
private_model.encrypt()
print("Model successfully encrypted:", private_model.encrypted)

Model successfully encrypted: True


### evaluation

In [ ]:
def take_samples_eval(data, n_samples=1000):
    """Returns images and labels based on sample size"""
    images, labels = [], []

    for i, d in enumerate(data):
        if i == n_samples:
            break
        image, label = d
        image = image.view(1, -1) # torch.Size([1, 784])
        images.append(image)
        label_one_hot = torch.nn.functional.one_hot(torch.tensor(label), 10)
        labels.append(label_one_hot)

    images = torch.cat(images)
    labels = torch.stack(labels)
    return images, labels
# test set
images_test_eval, labels_test_eval = take_samples_eval(data_test, n_samples=20)
images_test_enc_eval = crypten.cryptensor(images_test_eval, requires_grad=True)
labels_test_enc_eval = crypten.cryptensor(labels_test_eval, requires_grad=True)

In [ ]:
def test_plain_model(model, X, y):
    correct_count = 0
    criterion = crypten.nn.CrossEntropyLoss()

    model.eval()
    for i in range(len(X)):
      input_data = X[i]
      output = model(input_data)

      loss = criterion(output, y[i])
      loss.backward()

      grads = X.grad
      # print(grads)
      correct = (output[0].argmax().get_plain_text()).eq(y[i].get_plain_text())
      correct_count += correct.sum(0, keepdim=True).float()

    print("Accuracy: ", (correct_count.item() / len(y))*100)

    return grads

In [ ]:
grads_pl = test_plain_model(private_model, images_test_enc_eval, labels_test_enc_eval)

Accuracy:  100.0


### fgsm

In [ ]:
perturbed_images_eval = []
for i in range(len(images_test_enc_eval)):
    input = images_test_enc_eval[i]
    grad = grads_pl[i]
    perturbed_input = fgsm_attack(input, 0.01, grad)
    perturbed_images_eval.append(perturbed_input)

In [ ]:
adv_prediction = private_model(perturbed_images_eval[2].unsqueeze(0)).argmax()
adv_prediction.get_plain_text()

tensor([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]])